In [1]:

from river.base.classifier import Classifier as RiverClassifer
import queue
import pandas as pd
import typing
import inspect
import operator

In [2]:
import river
import river.datasets as datasets
from river import stream
from pprint import pprint
import os
from scipy.io import arff
import pandas as pd
from river import tree
from river import evaluate
from river import metrics

import numpy as np
from  streams.stream_section import StreamSection
from streams.utils import FL, FU, generate_stream_section
from my_datasets.getting_datasets import get_LED,get_Hyperplane,get_Electricity,get_AGRAWL,get_Airlines,get_CoverType,get_RandomRBF,transform_Airlines
from train_and_eval import train_and_evaluate
from functools import partial
from river.metrics.accuracy import Accuracy
from river.metrics import CohenKappa
from multiprocessing.pool import ThreadPool
from river.datasets.synth import ConceptDriftStream
from tqdm import tqdm
import matplotlib.pyplot as plt
# methods
from river.dummy import NoChangeClassifier,PriorClassifier
from river.cluster import CluStream
from river.forest.adaptive_random_forest import ARFClassifier
from river.tree.hoeffding_adaptive_tree_classifier import  HoeffdingAdaptiveTreeClassifier
from river.neighbors import KNNClassifier as riverKNN
from river.naive_bayes import GaussianNB
from semisupervised_methods.clustream_and_label import CluserAndLabel
from semisupervised_methods.incremental_classifier import IncrementalClassifer
from semisupervised_methods.one_nearest_neighbour import oneNNClassifer
from river.drift import ADWIN, PageHinkley
from river.base.classifier import Classifier as RiverClassifer
# from semisupervised_methods.my_method import MyClassifier 
import typing
from semisupervised_methods.ICLC import ICLC
from river.cluster import KMeans

In [3]:
# data = get_Airlines()
# airlines = transform_Airlines(data)

In [4]:
import logging
import datetime
now = datetime.datetime.now()
date_time = now.strftime("%d_%m_%Y_%H_%M_%S")
logging.basicConfig(filename=f'logs\\{date_time}.log', filemode='w',
                    format='%(asctime)s - %(message)s', level=logging.INFO, datefmt='%d-%b-%y %H:%M:%S')
logger = logging.getLogger()

In [5]:
stream = get_Hyperplane(n_features=10, n_drift_features=10, mag_change= 0.001,
                   noise_percentage = 0.01)
initial_stream = generate_stream_section(stream,'HyperPlane','synth',start=0,stop=10000)
probas = [0.1,0.2] 
Q=[(0,len(initial_stream.stream))]
HT1 = HoeffdingAdaptiveTreeClassifier
ARF1 = ARFClassifier
NB1= GaussianNB
incClasif2= IncrementalClassifer
incClasif1= IncrementalClassifer
incClasif3= IncrementalClassifer
m_params = [

  {'threshold':0.7,
  'classifier':ARF1,
  'params':{'seed':123}},
  {'threshold':0.7,
  'classifier':HT1,
  'params':{'seed':123}},
    {'threshold':0.7,
  'classifier':NB1,
  'params':{}}
]
delay_type =1
methods = [incClasif1,incClasif2,incClasif3]
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['ARF07','HT07','NB07'],
                         metric_fun=CohenKappa, delay_type=delay_type, B=50, K=10, delay=500,logger = logger)

HyperPlane_0_10000
HyperPlane_0_10000_constant_delay
HyperPlane_0_10000_constant_delay_ssl_0.1_0_10000
HyperPlane_0_10000_constant_delay_ssl_0.2_0_10000


In [6]:
stream = get_Hyperplane(n_features=10, n_drift_features=10, mag_change= 0.001,
                   noise_percentage = 0.01)
initial_stream = generate_stream_section(stream,'HyperPlane','synth',start=0,stop=10000)
Q=[(0,20000)]
probas = [0.1,0.2]  # probabilities of dropping the label
my_method = ICLC
Cl1= KMeans
ARF = ARFClassifier
m_params = [
 {  'classifer':ARF,
  'clustering_method':Cl1,
  'clustering_params':{},
  'classifier_params':{},
  'drift_detector': ADWIN,
  'nu':100}
]
methods = [my_method]
delay_type = 1 # constant delay

# # run the experiment 
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['ICLC'],
                         metric_fun=CohenKappa, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger)


HyperPlane_0_20000
HyperPlane_0_20000_constant_delay
HyperPlane_0_20000_constant_delay_ssl_0.1_0_20000
HyperPlane_0_20000_constant_delay_ssl_0.2_0_20000


In [8]:
stream = get_Hyperplane(n_features=10, n_drift_features=10, mag_change= 0.001,
                   noise_percentage = 0.01)
initial_stream = generate_stream_section(stream,'HyperPlane','synth',start=0,stop=10000)
Q=[(0,20000)]
probas = [0.1,0.2]  # probabilities of dropping the label
clustream = CluserAndLabel

m_params = [
    {'seed':123}]
methods = [clustream]
delay_type = 1 # constant delay

# # run the experiment 
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['CluStream'],
                         metric_fun=CohenKappa, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger)


HyperPlane_0_20000
HyperPlane_0_20000_constant_delay
HyperPlane_0_20000_constant_delay_ssl_0.1_0_20000
HyperPlane_0_20000_constant_delay_ssl_0.2_0_20000


In [ ]:
stream = get_CoverType()
initial_stream = generate_stream_section(stream,'CoverType','real',start=0,stop=len(stream))
Q=[(0,len(stream))]
probas = [0.1,0.2]  # probabilities of dropping the label

clustream = CluserAndLabel

m_params = [
    {'seed':123}]
methods = [clustream]
delay_type = 1 # constant delay

# # run the experiment 
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=m_params,
                         methods_name=['CluStream'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=1000,logger = logger)

In [8]:
# get the data stream
rbf = get_RandomRBF(change_speed=0.001, n_centroids=50)

# generate stream secion from it
initial_stream = generate_stream_section(datastream=rbf, stream_name='RBF_fast',
                                         dataset_type='synth', start=0, stop=2000)
Q = [(0, 2000)]  # stream secion boundries
probas = [0.1, 0.2]  # probabilities of dropping the label

onenn =oneNNClassifer

NB = GaussianNB
HT = HoeffdingAdaptiveTreeClassifier
ARF = ARFClassifier
incClasif = IncrementalClassifer
incClasif1 = IncrementalClassifer
incClasif2 = IncrementalClassifer
classificators_params = [
    {'threshold':0.5}]
methods = [onenn]
delay_type = 1 # constant delay

# run the experiment 
res = train_and_evaluate(initial_stream=initial_stream,  Q=Q,
                         probas=probas, methods=methods, methods_params=classificators_params,
                         methods_name=['1nn'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=5,logger = logger)


In [7]:
res

{(0,
  200): [('RBF_fast_0_200',
   {'1nn': ([Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0.00%,
      Accuracy: 0

In [ ]:
import logging
import datetime
now = datetime.datetime.now()
date_time = now.strftime("%d_%m_%Y_%H_%M_%S")
logging.basicConfig(filename=f'logs\\{date_time}.log', filemode='w',
                    format='%(asctime)s - %(message)s', level=logging.INFO, datefmt='%d-%b-%y %H:%M:%S')
logger = logging.getLogger()

In [ ]:
# get the data stream
rbf = get_RandomRBF(change_speed=0.001, n_centroids=50)

# generate stream secion from it
initial_stream = generate_stream_section(datastream=rbf, stream_name='RBF_fast',
                                         dataset_type='synth', start=0, stop=20000)
Q = [(0, 20000)]  # stream secion boundries
probas = [0.1, 0.2]  # probabilities of dropping the label

onenn =oneNNClassifer

NB = GaussianNB
HT = HoeffdingAdaptiveTreeClassifier
ARF = ARFClassifier
incClasif = IncrementalClassifer
incClasif1 = IncrementalClassifer
incClasif2 = IncrementalClassifer
classificators_params = [
    {'threshold':0.5}]
methods = [onenn]
delay_type = 1 # constant delay

# run the experiment 
res = train_and_evaluate(initial_stream=initial_stream, dataset_name='RBF_fast', Q=Q,
                         probas=probas, methods=methods, methods_params=classificators_params,
                         methods_name=['1nn'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=100, logger = logger)


In [ ]:
led3 = get_LED(0.1,True,3)
led5 = get_LED(0.1,True,5)
led7 = get_LED(0.1,True,7)

cds = ConceptDriftStream(ConceptDriftStream(led3,led5,position=3750,width = 3750),ConceptDriftStream(led5,led7,position=7500,width = 5000), position=7500,width=7500)
initial_stream = generate_stream_section(cds,'initail_LED_Drift_gradual','initail_LED_Drift_gradual',start=0,stop=20000)

In [ ]:
# get the data stream
rbf = get_RandomRBF(change_speed=0.0001, n_centroids=50)

# generate stream secion from it
initial_stream = generate_stream_section(datastream=rbf, stream_name='RBF_moderate',
                                         dataset_type='synth', start=0, stop=20000)
Q = [(0, 20000)]  # stream secion boundries
probas = [0.1, 0.2]  # probabilities of dropping the label

onenn =oneNNClassifer

NB = GaussianNB
HT = HoeffdingAdaptiveTreeClassifier
ARF = ARFClassifier
incClasif = IncrementalClassifer
incClasif1 = IncrementalClassifer
incClasif2 = IncrementalClassifer
classificators_params = [
    {'threshold': 0.7,
     'classifier': ARF,
     'params': {'seed':123}},
    {'threshold': 0.7,
     'classifier': NB,
     'params': {}},
         {'threshold': 0.7,
     'classifier': HT,
     'params': {'seed':123}}]
methods = [incClasif,incClasif1]
delay_type = 1 # constant delay

# run the experiment 
res = train_and_evaluate(initial_stream=initial_stream, dataset_name='RBF_moderate', Q=Q,
                         probas=probas, methods=methods, methods_params=classificators_params,
                         methods_name=['ARF07','NB07', 'HT07'],
                         metric_fun=Accuracy, delay_type=delay_type, B=50, K=10, delay=100)


In [4]:
dataset_name = 'LED_gradual'
Q=[(0,20000)]
probas = [0.5]
clustream = CluserAndLabel

m_params = [
    {'seed':123}]
methods = [clustream]
res = train_and_evaluate(initial_stream,dataset_name,Q,probas,methods,m_params,['Clustream'],Accuracy,1,B=50,K=10,delay=100)